In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import os
from freezegun import freeze_time

from src.email_manager import mock_send_email
from src.lambda_function import run_gen_training_week_process, run_update_training_week_process
from src.supabase_client import mock_upsert_training_week_update, mock_upsert_training_week_with_coaching
from src.supabase_client import list_users

In [45]:
def setup_mock_user():
    users = list_users()
    mock_user_id = int(os.environ["JAMIES_ATHLETE_ID"])

    mock_user = None
    for user in users:
        if user.athlete_id == mock_user_id:
            mock_user = user
            break

    print(f"Mock user: {mock_user}")
    return mock_user

In [46]:
mock_user = setup_mock_user()

@freeze_time("2024-08-21 20:00:00")
def mock_mid_week_update():
    run_gen_training_week_process(
        user=mock_user,
        upsert_fn=mock_upsert_training_week_with_coaching,
        email_fn=mock_send_email,
    )

@freeze_time("2024-08-18 20:00:00")
def mock_weekly_update():
    run_update_training_week_process(
        user=mock_user,
        upsert_fn=mock_upsert_training_week_update,
        email_fn=mock_send_email,
    )

Mock user: athlete_id=98390356 email='voynow99@gmail.com' preferences='A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays' created_at=datetime.datetime(2024, 8, 21, 21, 12, 55, 723179, tzinfo=datetime.timezone.utc)


In [50]:
mock_weekly_update()

No rates present in response headers


athlete_id=98390356 token still valid until 2024-08-23 20:21:00+00:00


AccessUnauthorized: Unauthorized: Authorization Error: [{'resource': 'Athlete', 'field': 'access_token', 'code': 'invalid'}]

In [49]:
mock_mid_week_update()

No rates present in response headers


athlete_id=98390356 token still valid until 2024-08-23 20:21:00+00:00


AccessUnauthorized: Unauthorized: Authorization Error: [{'resource': 'Athlete', 'field': 'access_token', 'code': 'invalid'}]